# Biomedical Data Bases, 2020-2021
### Programmatic Access to Databases
These are notes by prof. Davide Salomoni (d.salomoni@unibo.it) for the Biomedical Data Base course at the University of Bologna, academic year 2020-2021.

## Running an external script from Python

Here we demonstrate the use of the _subprocess_ module.

In [ ]:
# An example of running an external program (here: 'ls -l') from Python
import subprocess

sp = subprocess.run('ls -l', shell=True, capture_output=True, text=True)
print(sp.stdout)

### Running an external program and processing its output

Suppose we have a python program implementing an iterative method to compute the square root of that number, according to the following algorithm (called the _Heron's method_):

$$
x_0 = 1
\\
x_{n+1} = \frac{1}{2} (x_n + \frac{S}{x_n})
\\
\sqrt{S} = \lim_{n \to \infty} x_n
$$

If the program is called _sqrt_iterative.py_ (find it in the BDB github repo, or write one yourself) and expects in input the number for which we want to compute the square root and the number of iterations to perform, we could call it from Jupyter with the usual `!` character like this:

In [ ]:
! python3 sqrt_iterative.py 23941 9

However, what if we want to capture its output from a Python program? We can use the `subprocess` module like this:

In [ ]:
import subprocess
import math

number = 23941
iterations = 9

sp = subprocess.run('python3 sqrt_iterative.py %s %s' % (number, iterations), shell=True, capture_output=True, text=True)
result = float(sp.stdout.strip()) # what happens if the result cannot be converted to a float? try!

print('After %d iterations, the square root of %d is %f' % (iterations, number, result))
print('math.sqrt(%d) returns %f' % (number, math.sqrt(number)))
print('The difference with math.sqrt(%d) is %f' % (number, result-math.sqrt(number)))

## Using the _requests_ module

In [ ]:
# test the requests module querying Google
import requests
res = requests.get('http://www.google.com')
print(res)

In [ ]:
print(res.status_code)

In [ ]:
# if the requests call succeeded, print the text that was returned.
res = requests.get('http://www.google.com')
if res.status_code == 200:
    print(res.text)

### Remember to check the status code

In [ ]:
r = requests.get('https://github.com/timelines.json')
print(r.status_code)

In [ ]:
print(r.text)

### Query PDB using REST calls

Note that when the output is returned in JSON format we can easily parse it using normal Python dictionaries and lists.

In [ ]:
# query the PDB using the REST API. It returns JSON output.
r = requests.get('https://data.rcsb.org/rest/v1/core/entry/4GYD')

# convert the json return value to a Python dictionary
data = r.json()

# check it is indeed a dictionary
type(data)

In [ ]:
# since 'data' is a dictionary, check what are its keys:
data.keys()

In [ ]:
# get info from the 'cell' key:
data['cell']

In [ ]:
# get info for polymer entity data, providing PDB ID and polymer ID:
r = requests.get('https://data.rcsb.org/rest/v1/core/polymer_entity/4GYD/1')
data = r.json()
data.keys()

In [ ]:
# see what's inside the 'entity_poly' key:
data['entity_poly']

In [ ]:
# get annotations
r2 = requests.get('https://data.rcsb.org/rest/v1/core/pubmed/4GYD')
data2 = r2.json()
data2.keys()

In [ ]:
# this is the pubmed abstract:
data2['rcsb_pubmed_abstract_text']

### Chemical component

In [ ]:
r = requests.get('https://data.rcsb.org/rest/v1/core/chemcomp/CFF')
data = r.json()

In [ ]:
data.keys()

In [ ]:
data['chem_comp']

### Drug Bank

In [ ]:
r = requests.get('https://data.rcsb.org/rest/v1/core/drugbank/CFF')
data = r.json()
data.keys()

In [ ]:
data['drugbank_info'].keys()

In [ ]:
data['drugbank_info']['description']

In [ ]:
data['drugbank_info']['indication']

### Processing multiple files

In [ ]:
protein_ids = ['4GYD', '4H0J', '4H0K']

protein_dict = dict()
for protein in protein_ids:
    r = requests.get('https://data.rcsb.org/rest/v1/core/entry/%s' % protein)
    data = r.json()
    protein_dict[protein] = data['cell']

# now print e.g. length_a, length_b and length_c for all the proteins:
for (key,value) in protein_dict.items():
    print('Protein %s: a=%f, b=%f, c=%f' % (key, value['length_a'], value['length_b'], value['length_c']))

### Getting sequence data in FASTA format

Note that in this case the output is returned as regular text, i.e. it is not JSON-formatted.

In [ ]:
# print FASTA data for some proteins
protein_ids = ['4GYD', '4H0J', '4H0K']

for protein in protein_ids:
    r = requests.get('https://www.rcsb.org/fasta/entry/%s/download' % protein)
    print(r.text)

## The PDB Search API

In [ ]:
# a BLAST-like example using the PDB search API
fasta = "MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLPARTVETRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKLNPPDESGPGCMNCKCVIS"
my_query = '''{
    "query": {
        "type" : "terminal",
        "service" : "sequence",
        "parameters" : {
            "evalue_cutoff" : 1,
            "identity_cutoff" : 0.9,
            "target" : "pdb_protein_sequence",
            "value" : "%s"
        }
    },
    "request_options" : {
        "scoring_strategy" : "sequence"
    },
    "return_type" : "polymer_entity"
}''' % fasta
r = requests.get('http://search.rcsb.org/rcsbsearch/v1/query?json=%s' % requests.utils.requote_uri(my_query))
j = r.json()

In [ ]:
# these are keys of the dictionary:
print(j.keys())

In [ ]:
# let's print the results:
print("We got %s matches" % j['total_count'])
print("The first %s results follow:" % len(j['result_set']))
for item in j['result_set']:
    print(item['identifier'], "score =", item['score'])

In [ ]:
# a sequence motif search
# we use here the Zinc finger Cys2His2-like fold group
# its PROSITE signature is available at https://prosite.expasy.org/PS00028
my_query = '''
{
  "query": {
    "type": "terminal",
    "service": "seqmotif",
    "parameters": {
      "value": "C-x(2,4)-C-x(3)-[LIVMFYWC]-x(8)-H-x(3,5)-H",
      "pattern_type": "prosite",
      "target": "pdb_protein_sequence"
    }
  },
  "return_type": "polymer_entity"
}
'''
r = requests.get('http://search.rcsb.org/rcsbsearch/v1/query?json=%s' % requests.utils.requote_uri(my_query))
j = r.json()

In [ ]:
j.keys()

In [ ]:
print("There are %s results in total, we got back details for the first %s" % 
      (j['total_count'], len(j['result_set'])))

In [ ]:
print('This is the detailed info for the first result:')
print(j['result_set'][0])
print('\nThe identifiers for the returned results are:')
for item in j['result_set']:
    print(item['identifier'])

## GraphQL 

In [ ]:
# a GraphQL query
my_query = '''
{
    entry(entry_id: "4GYD") {
        cell {
            Z_PDB
            angle_alpha
            angle_beta
            angle_gamma
            formula_units_Z
            length_a
            length_b
            length_c
            pdbx_unique_axis
            volume
        }
    }
}
'''

r = requests.get('https://data.rcsb.org/graphql?query=%s' % requests.utils.requote_uri(my_query))
j = r.json()

In [ ]:
# check the keys of the dictionary:
j.keys()

In [ ]:
# explore what is in j['data']:
j['data']

In [ ]:
# print results with some formatting:
params = j['data']['entry']['cell']
for key,value in params.items():
    print(key, ':', value)

## Uniprot

In [ ]:
# get data from Uniprot using the Proteins API
# note that we are returned a list, not a dictionary
requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=10&accession=P0A3X7&reviewed=true"

# note that we must specify that we want JSON output using the 'headers' parameter to requests.get()
r = requests.get(requestURL, headers={"Accept" : "application/json"})
j = r.json()
type(j)

In [ ]:
# the returned list holds the entries we asked for
# note that there is only one entry:
len(j)

In [ ]:
# this one entry is actually a dictionary:
j[0]

In [ ]:
# check the keys of the dictionary j[0]
j[0].keys()

In [ ]:
# these are the entries in the key 'dbReferences'
j[0]['dbReferences']

In [ ]:
# print the data we were looking for:
print("Data for accession %s (ID: %s)" % (j[0]['accession'], j[0]['id']))
print("List of Gene Ontologies:")
for item in j[0]['dbReferences']:
    if item['type'] == "GO":
        print("  id: %s, term: %s, source: %s" % (
                item['id'],
                item['properties']['term'],
                item['properties']['source']))


## NCBI

In [ ]:
headers = {'Accept': 'application/json'}
r = requests.get('https://api.ncbi.nlm.nih.gov/datasets/v1alpha/gene/id/%s' % 8291, headers=headers)
j = r.json()
j

In [ ]:
gene = j['genes'][0]['gene']
gene['description']